# 첫번째 프로젝트

## 프로젝트의 주제와 목적
최근 북미, 유럽, 일본, 그외 지역에서의 게임 판매량 데이터를 분석하여 다음 출시할 게임의 방향을 정하는것이 목적이다.  
정확한 분석을 위해서는 게임의 판매량에 영향을 미치는요인이 무엇인지 알아내야한다.  
즉 판매량과 장르, 플렛폼, 출시년도의 연관성을 따져보아야 한다.

## 우선 분석과정을 4개의 작은 단위로 나눠볼 수 있다
1. 지역별 선호하는 장르가 다른가?
2. 연도별 트랜드가 있는가?
3. 플랫폼에따른 판매량의 차이가 있는가?
4. 판매량이 가장 높은 게임들은 무슨 특징을 가지고 있는가?

이 분석과정을 통해, **"잘 팔리는"** 게임을 만들기위해 어떤 부분을 우선시 해야하는지를 판단해야 한다


## 지역별 선호하는 장르가 다른가?  
선호하는 장르란?  "많이 팔리는 장르"이다.  
한마디로 **"지역별 판매량과, 장르가 서로 관련이 있는가?"** 를 확인해야한다  

만약 장르가 두가지 뿐이라면 어떤 장르가 더 잘팔리는지를 비교하면 된다
하지만 수많은 장르와 수만개이상의 게임을 비교한다면... 한번에 정확한 비교를 할 수 없다  

그러므로, 통계학적 방법을 이용하여 데이터를 **단순화** 시켜, 효율적이고 정확한 비교를 해야한다  

단순화한다는게 무슨뜻일까?  
예를들면 사람의 외모나 능력, 습관을 가지고 "이사람은 예의바른사람인가?" 를 판단하긴 힘들다.  
하지만 그 사람이 초면에 반말을하는 습관이있다면, 예의없는 사람일 확률이 매우높다.  
즉, 외모나 능력보다 반말을한다는 조건이 비교적 확실한 판단 기준이 될 수 있다.  
동일하게, 데이터의 특징들 중 비교적 확실한 판단기준이되는 특징을 찾고(경우에따라 특징들을 혼합하기도한다) 그것을 기준으로 나누는것이 단순화라고 볼 수 있다.

In [47]:
import pandas as pd


# 데이터 불러오기
df = pd.read_csv('/content/drive/MyDrive/AI_bootcamp/vgames2.csv', encoding='CP949')

# 인덱스제거
df = df.drop('Unnamed: 0', axis=1)

# 데이터 결측치 확인
df.isnull().sum()

Name             0
Platform         0
Year           271
Genre           50
Publisher       58
NA_Sales         0
EU_Sales         0
JP_Sales         0
Other_Sales      0
dtype: int64

In [50]:
# 데이터 단위를 (0.00) 형태로 통일하는 함수선언
def unification(str):
    if ('K' in str) == True:
        remove_k = str.replace('K','')
        return float(remove_k)*0.001
    elif ('M' in str) == True:
        remove_m = str.replace('M','')
        return float(remove_m)

# 단위통일 컬럼에 적용하는 함수
def dfUnification(df):
    df = df.apply(unification)

# 각 컬럼 단위통일
dfUnification(df['NA_Sales'])
dfUnification(df['JP_Sales'])
dfUnification(df['EU_Sales'])
dfUnification(df['Other_Sales'])